La première chose à faire est d'installer graphkit-learn(https://github.com/jajupmochi/graphikit-learn). Pour cela il suffit de taper:

pip install graphkit-learn



In [94]:
from gklearn.utils.graphfiles import loadDataset
import networkx as nx
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np

from __future__ import print_function
import torch

import sys
sys.path.append('/home/public/REC_FORMES/gedlibpy')
import librariesImport
import gedlibpy

In [121]:
Gs,y = loadDataset('/home/public/REC_FORMES/MAO/dataset.ds')



def label_to_color(label):
    if label == 'C':
        return 0.1
    elif label == 'O':
        return 0.8
    
def nodes_to_color_sequence(G):
    return [label_to_color(c[1]['label'][0]) for c in G.nodes(data=True)]


#for e in Gs[13].edges():
#    print(Gs[13][e[0]][e[1]]['bond_type'])
print('edge max label',max(max([[G[e[0]][e[1]]['bond_type'] for e in G.edges()] for G in Gs])))
G1 = Gs[7]
G2 = Gs[1]
print(y[1],y[9])
plt.figure(0)
nx.draw_networkx(G1,with_labels=True,node_color = nodes_to_color_sequence(G1),cmap='autumn')

plt.figure(1)
nx.draw_networkx(G2,with_labels=True,node_color = nodes_to_color_sequence(G2),cmap='autumn')



def build_node_dictionnary(GraphList):
    #extraction de tous les labels d'atomes
    node_labels=[]
    for G in Gs:
        for v in nx.nodes(G):
            if not G.nodes[v]['label'][0] in node_labels:
                node_labels.append(G.nodes[v]['label'][0])
    node_labels.sort()
    #extraction d'un dictionnaire permettant de numéroter chaque label par un numéro.
    dict={}
    k=0
    for label in node_labels:
        dict[label]=k
        k=k+1
    print(node_labels)
    print(dict,len(dict))
    return dict,max(max([[int(G[e[0]][e[1]]['bond_type']) for e in G.edges()] for G in GraphList]))


                   
dict,nb_edge_labels=build_node_dictionnary(Gs)
print('dict=',dict)
print('nb_edge_labels=',nb_edge_labels)

#A1=nx.to_scipy_sparse_matrix(G1,dtype=int,weight='bond_type').todense()
lab1=[dict[G1.nodes[v]['label'][0]] for v in nx.nodes(G1)]
nb_node1=nx.number_of_nodes(G1)#A1.shape[0]
print('nb nodes:',nb_node1,G1.number_of_nodes())

#A2=nx.to_scipy_sparse_matrix(G2,dtype=int,weight='bond_type').todense()
lab2=[dict[G2.nodes[v]['label'][0]] for v in nx.nodes(G2)]
nb_node2=nx.number_of_nodes(G2)#A2.shape[0]

#definition des coûts de substitution/insertion/suppression.

node_cost=np.full((len(dict),len(dict)),1)
node_del=2
node_ins=2
node_cost[np.diag_indices(len(dict))]=0
                   
edge_cost=np.full((nb_edge_labels+1,nb_edge_labels+1),.5)
edge_del=1
edge_ins=1
edge_cost[np.diag_indices(nb_edge_labels+1)]=0
 











plt.show()

edge max label 3
0.0 0.0
['C', 'N', 'O']
{'C': 0, 'N': 1, 'O': 2} 3
dict= {'C': 0, 'N': 1, 'O': 2}
nb_edge_labels= 3
nb nodes: 16 16


En utilisant les tableaux node_cost, node_ins et node_del completez la fonction cost_node qui remplie une matrice de taille $(n+1)\times (m+1)$ (n,m taille de G1 et G2) contenant les coûts d'affectation, de suppression et d'insertion des sommets.

In [116]:
def cost_node(G1,G2):
    nb_node1=G1.number_of_nodes()
    nb_node2=G2.number_of_nodes()

    C=np.full((nb_node1+1,nb_node2+1),0)

    C[-1][:-1] = node_ins
    
    for line in C[:-1]:
        line[-1] = node_del
    
    for i in range(nb_node1):
        for j in range(nb_node2):
            C[i][j] = node_cost[dict[G1.nodes[i]['label'][0]]][dict[G2.nodes[j]['label'][0]]]
            
    # To be completed
    return C

print(cost_node(G1, G2))

[[0 1 1 1 1 1 1 1 1 1 0 1 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [0 1 1 1 1 1 1 1 1 1 0 1 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [1 0 0 0 0 0 0 0 0 0 1 0 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 0]]


Completez le code de la fonction cost_map_edge qui renvoie le coût d'appariement optimal entre les arêtes incidentes au sommet i de G1 et le sommet j de G2. On utilisera pour cela les fonctions de coût. On pourra commencer par initialiser deux listes edge_i et edge_j contenant les labels des arêtes incidentes à i et j. 

In [ ]:
def cost_map_edge(G1,G2,i,j,edge_cost,edge_del,edge_ins):
    edge_i = []
    edge_j = []
    
    # initialisation of both lists edge_i and edge_j
    neighbors_i = G1.adj[i]
    neighbors_j = G2.adj[j]
    
    for neighbor in neighbors_i.values():
        edge_i.append(int(neighbor['bond_type']))
    for neighbor in neighbors_j.values():
        edge_j.append(int(neighbor['bond_type']))
        
        
    C=np.full((len(edge_i)+1,len(edge_j)+1),0, dtype='float')
    
    #initialization of C. To be completed.
    C[-1, :-1] = edge_ins
    C[:-1, -1] = edge_del
        
    for first_edge_index in range(len(edge_i)):
        for second_edge_index in range(len(edge_j)):
            C[first_edge_index, second_edge_index] = edge_cost[edge_i[first_edge_index], edge_j[second_edge_index]]
    
    #application of the Hungarian algortithm on C.
    result=gedlibpy.hungarian_LSAPE(C)
    
    cost=np.array(result[2]).sum()+np.array(result[3]).sum()
    
    return cost

cost_map_edge(G1, G2, 1, 1, edge_cost, edge_del, edge_ins)

Completez le code de la fonction cost_edge qui renvoie une matrice $(n+1)\times (m+1)$ (n,m : taille des deux graphes) codant le coût d'appariement des paires de sommets uniquement en termes d'arêtes. 

In [117]:
def cost_edge(G1,G2,edge_cost,edge_del,edge_ins):
    nb_node1=G1.number_of_nodes()
    nb_node2=G2.number_of_nodes()
    C=np.full((nb_node1+1,nb_node2+1),0)

    
    G1_edges = G1.edges
    G2_edges = G2.edges
    
    edges_bond_type_G1 = []
    edges_bond_type_G2 = []
    
    for g1 in G1_edges.values():
        edges_bond_type_G1.append(int(g1['bond_type']))
    for g2 in G2_edges.values():
        edges_bond_type_G2.append(int(g2['bond_type']))
    
    for j in range(nb_node2):
        C[-1,j] = edge_ins*G2.degree(j)
    for i in range(nb_node1):
        C[i,-1] = edge_del*G1.degree(i)
    
    for e1 in range(nb_node1):
        for e2 in range(nb_node2):
            C[e1, e2] = cost_map_edge(G1,G2,e1,e2,edge_cost,edge_del,edge_ins)
        
    return C

cost_edge(G1, G2, edge_cost, edge_del, edge_ins)

array([[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2],
       [1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 2, 3],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2],
       [1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 2, 3],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2],
       [1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 2, 3],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2],
       [1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 2, 3],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2],
       [1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 0, 1],
       [1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 0, 1],
       [2, 3, 2, 3, 2, 2, 2, 2, 3, 2, 2, 1, 0]])

On donne ci-dessous le code de la fonction compute_graph_mapping qui calcule un mapping à partir d'une matrice de coûts. Le premier tableau renvoyé indique le mapping des élements du premier ensemble vers le second. Le deuxième tableau indique le mapping des éléments du second ensemble vers le premier le dernier argument indique le coût de l'appariement. 

A partir de cette fonction calculez le coût de l'appariement optimal induit par la matrice de coût node_cost+$\frac{1}{2}$edge_cost. Le coût retourné est une borne inférieure de la ged (cf cours).

In [118]:
def compute_graph_mapping(C):
    result=gedlibpy.hungarian_LSAPE(C)
    cost=np.array(result[2]).sum()+np.array(result[3]).sum()

    return result[0],result[1],cost

#to be completed
edge_cost = cost_edge(G1,G2,edge_cost,edge_del,edge_ins)
node_cost = cost_node(G1,G2)

C = node_cost + (1/2) * edge_cost
rho,var_rho,lower_bound=compute_graph_mapping(C)
print('lower bound =',lower_bound)

lower bound = 12.0


Définissez le code de la fonction from_mapping_function_to_matrix qui renvoie une matrice d'$\epsilon$-assignement à partir des deux fonctions rho et varrho. 

Calculez l'appariement optimal (cout+fonctions) pour une matrice de coût égale au coût sur les noeuds + le coût sur les arêtes. Utilisez la fonction ci-dessus pour en déduire le coût de la ged uniquement en termes de sommets. 

In [119]:
def from_mapping_function_to_matrix(rho,var_rho):
    n=len(rho)
    m=len(var_rho)
    X=np.zeros((n+1,m+1))

    for i in range(n):
        X[i, int(rho[i])] = 1
    
    for k in range(m):
        X[int(var_rho[k]), k] = 1
        
    return X

from_mapping_function_to_matrix(rho, var_rho)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 

Définissez le code de la fonction ged_edge qui calcule le coût de la ged uniquement en termes d'arêtes à partir des appariements définis par rho et varrho. Le principe est le suivant:
1. Pour toute arête (i,j) de G1
    * Soient (k,l) les images de i et j par rho.
    * Si k ou l ne correspondent pas à un sommet de G2, ajouter un coût de suppression d'arête
    * Sinon, si $(k,l) \in E_2$  ajouter un coût de substitution
    * Sinon si $(k,l)\not\in E_2$ ajouter un coût de suppression
2. Pour toute arête (k,l) de G2
    * Soient (i,j) les images de (k,l) par varrho
    * Si i ou j ne correspondent pas à un sommet de G1, ajouter un coût d'insertion.
    * Si $(i,j)\not\in E_1$, ajouter un coût d'insertion.

Utilisez cette fonction et le calcul précédent pour calculer une borne supérieure de la distance d'édition. 

In [120]:
ged = 0

def ged_edge(G1,G2,rho,varrho,edge_cost,edge_del,edge_ins):
    edge_ged=0
    G1_nodes = G1.nodes()
    G2_nodes = G2.nodes()
    
    G1_edges = nx.edges(G1)
    G2_edges = nx.edges(G2)
    
    for e1 in G1_edges:
        i, j = e1[0], e1[1]
        k, l = rho[i], rho[j]
        if k not in G2_nodes or l not in G2_nodes:
            edge_ged += edge_del
        elif (k,l) in G2_edges:
            edge_ged += edge_cost[int(G1.edges[i, j]['bond_type']), int(G2.edges[k, l]['bond_type'])]
        elif (k,l) not in G2_edges:
            edge_ged += edge_del
            
        
    for e2 in nx.edges(G2):
        k, l = e2[0], e2[1]
        i, j = varrho[k], varrho[l]
        if i not in G1_nodes or j not in G1_nodes:
            edge_ged += edge_ins
        elif (i,j) not in G1_edges:
            edge_ged += edge_ins
            
    return edge_ged

ged+=ged_edge(G1,G2,rho,var_rho,edge_cost,edge_del,edge_ins)
print('ged =',ged)

edge_cost = cost_edge(G1,G2,edge_cost,edge_del,edge_ins)
node_cost = cost_node(G1,G2)

C = node_cost + edge_cost
rho,var_rho,cost=compute_graph_mapping(C)
X = from_mapping_function_to_matrix(rho, var_rho)

upper_bound = (X*node_cost).sum() + ged_edge(G1,G2,rho,var_rho,edge_cost,edge_del,edge_ins)
print("upper bound =", upper_bound)

ged = 4
upper bound = 12.0
